In [1]:
import pandas as pd
import numpy as np

## 과제 1: groupby

In [2]:
transactions = pd.read_csv('./data/retail/transactions.csv', parse_dates=['date'])
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [3]:
(
    transactions
    .groupby('store_nbr')[['transactions']]
    .sum()
    .sort_values('transactions', ascending=False)
).iloc[:10]

,transactions
store_nbr,
44,7273093
47,6535810
45,6201115
46,5990113
3,5366350
48,5107785
8,4637971
49,4574103
50,4384444


## 과제 2: groupby
Multiple Columns

In [4]:
transactions['month'] = transactions['date'].dt.month
transactions.head()

,date,store_nbr,transactions,month
0,2013-01-01,25,770,1
1,2013-01-02,1,2111,1
2,2013-01-02,2,2358,1
3,2013-01-02,3,3487,1
4,2013-01-02,4,1922,1


In [5]:
(
    transactions
    .groupby(['store_nbr', 'month'])[['transactions']]
    .sum()
    .sort_values(['month', 'transactions'], ascending=[True, False])
)

,,transactions
store_nbr,month,
44,1,628438
47,1,568824
45,1,538370
46,1,522763
3,1,463260
...,...,...
32,12,86167
21,12,84128
42,12,76741


## 과제 3: 데이터프레임 멀티인덱스

In [6]:
grouped = (
    transactions
    .groupby(['store_nbr', 'month'])
    .agg({'transactions': ['sum', 'mean']})
    .sort_values(['month', ('transactions', 'sum')], ascending=[True, False])
)
grouped.head()

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571

In [7]:
grouped.loc[[(3, 1)]]

transactions             
                         sum         mean
store_nbr month                          
3         1           463260  3151.428571

In [8]:
grouped.loc[:, [('transactions', 'mean')]]

,,transactions
,,mean
store_nbr,month,
44,1,4246.202703
47,1,3843.405405
45,1,3637.635135
46,1,3532.182432
3,1,3151.428571
...,...,...
32,12,718.058333
21,12,1402.133333


In [9]:
(
    grouped
    .droplevel(0, axis=1)
    .reset_index()
)

,store_nbr,month,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
...,...,...,...,...
636,32,12,86167,718.058333
637,21,12,84128,1402.133333
638,42,12,76741,1279.016667
639,29,12,76627,1277.116667


## 과제 4: Agg 메서드

In [10]:
transactions = (
    transactions
    .assign(
        target_pct=transactions['transactions'] / 2500,
        met_target=(transactions['transactions'] / 2500) >= 1,
        bonus_payable=((transactions['transactions'] / 2500) >= 1) * 100,
        month=transactions.date.dt.month,
        day_of_week=transactions.date.dt.dayofweek
    )
)
transactions

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2
...,...,...,...,...,...,...,...,...
83483,2017-08-15,50,2804,8,1.1216,True,100,1
83484,2017-08-15,51,1573,8,0.6292,False,0,1
83485,2017-08-15,52,2255,8,0.9020,False,0,1
83486,2017-08-15,53,932,8,0.3728,False,0,1


In [11]:
(
    transactions
    .groupby('store_nbr')
    .agg({
        'met_target': 'mean',
        'bonus_payable': 'sum'
    })
    .sort_values('bonus_payable', ascending=False)
)

,met_target,bonus_payable
store_nbr,,
47,0.999404,167600
44,0.998807,167500
45,0.997615,167300
3,0.998210,167300
46,0.989267,165900
8,0.888425,148900
48,0.690519,115800
49,0.637448,106900
50,0.453190,76000


In [12]:
(
    transactions
    .groupby('day_of_week')
    .agg({
        'met_target': 'mean',
        'bonus_payable': 'sum'
    })
    .sort_values('bonus_payable', ascending=False)
)

,met_target,bonus_payable
day_of_week,,
5,0.222204,266400
6,0.204001,241700
4,0.179007,213000
0,0.160214,191600
2,0.160572,191000
1,0.146299,175500
3,0.142077,169100


## 과제 5: transform 메서드

In [13]:
transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [14]:
(
    transactions
    .query('store_nbr == 25')
    .groupby('day_of_week')
    .agg({'transactions': 'mean'})
)

,transactions
day_of_week,
0,808.830435
1,740.245690
2,849.978261
3,800.818182
4,1102.817391
5,1376.090517
6,909.604348


In [15]:
(
    transactions
    .assign(
        avg_store_transactions=(
            transactions
            .groupby(['store_nbr', 'day_of_week'])['transactions']
            .transform('mean')
        ),
        difference=lambda x: x['transactions'] - x['avg_store_transactions']
    )
)

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,avg_store_transactions,difference
0,2013-01-01,25,770,1,0.3080,False,0,1,740.245690,29.754310
1,2013-01-02,1,2111,1,0.8444,False,0,2,1870.782427,240.217573
2,2013-01-02,2,2358,1,0.9432,False,0,2,1952.652720,405.347280
3,2013-01-02,3,3487,1,1.3948,True,100,2,3142.682008,344.317992
4,2013-01-02,4,1922,1,0.7688,False,0,2,1499.569038,422.430962
...,...,...,...,...,...,...,...,...,...,...
83483,2017-08-15,50,2804,8,1.1216,True,100,1,2342.410788,461.589212
83484,2017-08-15,51,1573,8,0.6292,False,0,1,1548.448133,24.551867
83485,2017-08-15,52,2255,8,0.9020,False,0,1,1892.588235,362.411765
83486,2017-08-15,53,932,8,0.3728,False,0,1,877.214286,54.785714
